In [24]:
import matplotlib
import matplotlib.pyplot as plt
import pysam
import pandas as pd
import numpy as np
import os
import collections
import json
import ast
from pygenomeviz import GenomeViz

# Import DataFrames

In [25]:
smnDF1 = pd.read_csv('/home/mark/Desktop/SMN/Dataframes/FullHaplotype_RepeatMaskerFilled.csv', low_memory=False).drop(columns=['Unnamed: 0'])

In [26]:
smnDF1[(smnDF1['15']=='HG03248.hap2') & (smnDF1['16'].str.contains('SMN',regex=True))]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
2499617,0.2,0.0,0.0,haplotype2-0000164,72891830,72892406,(111032687),C,SMN1_EXON9,Unspecified,(0),577,1,1507852,NaN,HG03248.hap2,SMN2
2499649,1.9,0.0,0.0,haplotype2-0000164,72892851,72892904,(111032189),C,SMN1_EXON8,Unspecified,(0),54,1,1507853,NaN,HG03248.hap2,SMN2
2499858,0.0,0.0,0.0,haplotype2-0000164,72898673,72898783,(111026310),C,SMN1_EXON7,Unspecified,(0),111,1,1507861,NaN,HG03248.hap2,SMN2
2499932,0.0,0.0,0.0,haplotype2-0000164,72900096,72900191,(111024902),C,SMN1_EXON6,Unspecified,(0),96,1,1507864,NaN,HG03248.hap2,SMN2
2499989,0.0,0.0,0.0,haplotype2-0000164,72901979,72902131,(111022962),C,SMN1_EXON5,Unspecified,(0),153,1,1507866,NaN,HG03248.hap2,SMN2
2500005,0.5,0.0,0.0,haplotype2-0000164,72902291,72902491,(111022602),C,SMN1_EXON4,Unspecified,(0),201,1,1507867,NaN,HG03248.hap2,SMN2
2500039,0.0,0.0,0.0,haplotype2-0000164,72903341,72903460,(111021633),C,SMN1_EXON3,Unspecified,(0),120,1,1507868,NaN,HG03248.hap2,SMN2
2500130,0.0,0.0,0.0,haplotype2-0000164,72905939,72906010,(111019083),C,SMN1_EXON2,Unspecified,(0),72,1,1507870,NaN,HG03248.hap2,SMN2
2500666,0.0,0.0,0.0,haplotype2-0000164,72919665,72919762,(111005331),C,SMN1_EXON1,Unspecified,(0),98,1,1507886,NaN,HG03248.hap2,SMN2


In [27]:
#smnDF[(smnDF['15']=='HG02666.hap1') & (smnDF['16'].str.contains('NAIP',regex=True))]

In [28]:
goodRows=[]
for sampleHap in set(smnDF1['15']):
    testDF = smnDF1[smnDF1['15']==sampleHap].copy()
    #The key is only looking at contigs with SMN there are some contigs with the other genes that get filtered out.
    allowableContigs = set(testDF[testDF['16'].str.contains('SMN', regex=True)]['3'])
    #print(allowableContigs)
    if len(allowableContigs)>1:
        #print(sampleHap)
        continue
    else:
        for row in testDF[testDF['3'].isin(allowableContigs)].index:
            goodRows.append(row)

In [29]:
smnDF= smnDF1.loc[goodRows].copy()

In [30]:
smnDF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
342598,8.1,0.0,1.0,haplotype2-0000116,69686689,69686984,(111614129),+,AluY,SINE/Alu,1,293,(18),4927786,NaN,HG00513.hap2,SINE/Alu
342608,16.3,5.3,3.7,haplotype2-0000116,69686985,69687365,(111613748),C,L1MB8,LINE/L1,(473),5705,5331,4927787,NaN,HG00513.hap2,LINE/L1
342618,8.1,0.3,3.4,haplotype2-0000116,69687366,69687671,(111613442),C,AluY,SINE/Alu,(14),297,1,4927788,NaN,HG00513.hap2,SINE/Alu
342627,16.3,5.3,3.7,haplotype2-0000116,69687672,69687701,(111613412),C,L1MB8,LINE/L1,(848),5330,5305,4927789,NaN,HG00513.hap2,LINE/L1
342628,14.8,2.1,1.8,haplotype2-0000116,69687702,69687984,(111613129),C,AluJb,SINE/Alu,(0),302,19,4927790,NaN,HG00513.hap2,SINE/Alu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333960,22.1,7.8,2.6,haplotype2-0000073,72105047,72105748,(110470618),+,MLT1A0,LTR/ERVL-MaLR,1,738,(633),4086904,NaN,HG03009.hap2,LTR/ERVL-MaLR
2333984,23.0,2.1,5.0,haplotype2-0000073,72105735,72105909,(110470457),+,MLT1A0,LTR/ERVL-MaLR,819,1029,(460),4086905,*,HG03009.hap2,LTR/ERVL-MaLR
2333997,19.9,2.2,1.9,haplotype2-0000073,72106149,72106558,(110469808),+,MLT1A0,LTR/ERVL-MaLR,1325,1735,(0),4086906,NaN,HG03009.hap2,LTR/ERVL-MaLR
2334013,23.4,3.9,1.6,haplotype2-0000073,72106559,72106915,(110469451),+,MLT1A0,LTR/ERVL-MaLR,1,365,(0),4086907,NaN,HG03009.hap2,LTR/ERVL-MaLR


In [31]:
with open('/home/mark/Desktop/SMN/json/SMN.json') as json_file:
    smnData = json.load(json_file)

with open('/home/mark/Desktop/SMN/json/SERF1A.json') as json_file:
    serfData = json.load(json_file)
    
with open('/home/mark/Desktop/SMN/json/NAIP.json') as json_file:
    naipData = json.load(json_file)
    
with open('/home/mark/Desktop/SMN/json/GTF2H2.json') as json_file:
    gtfData = json.load(json_file)

In [32]:
naipData

{'NA19705_hap1': {'0': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype1-0000025:110488194-110523856'},
  '1': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype1-0000025:110599445-110604125'},
  '2': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype1-0000025:110615221-110620335'},
  '3': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype1-0000025:110918081-110922760'},
  '4': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype1-0000025:110933856-110969522'},
  '5': {'Length': 'FL',
   'Coordinates': 'haplotype1-0000025:111680483-111737620'}},
 'HG00096_hap2': {'0': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype2-0000101:110444052-110468049'},
  '1': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype2-0000101:110547448-110552127'},
  '2': {'Length': 'TRUNCATED',
   'Coordinates': 'haplotype2-0000101:110563215-110568354'},
  '3': {'Length': 'FL',
   'Coordinates': 'haplotype2-0000101:111063029-111120181'}},
 'NA19836_hap1': {'0': {'Length': 'TRUNCATED',
   'Coordinat

In [33]:
sampleHaplotypeDict={}
for sample in set(smnDF['15']):
    testDF = smnDF[smnDF['15']==sample].copy()
    haplotype = [x for x in testDF['3']][0]
    sampleHaplotypeDict[sample]=haplotype

In [34]:
sampleHaplotypeDict

{'HG00513.hap2': 'haplotype2-0000116',
 'NA19650.hap2': 'haplotype2-0000085',
 'HG00733.hap2': 'haplotype2-0000085',
 'HG00514.hap1': 'haplotype1-0000032',
 'HG04217.hap2': 'haplotype2-0000057',
 'HG03248.hap2': 'haplotype2-0000164',
 'HG02666.hap2': 'haplotype2-0000067',
 'HG02492.hap1': 'haplotype1-0000027',
 'HG02554.hap2': 'haplotype2-0000163',
 'NA19331.hap1': 'haplotype1-0000049',
 'NA19384.hap1': 'haplotype1-0000039',
 'NA19129.hap1': 'haplotype1-0000007',
 'NA19983.hap1': 'haplotype1-0000001',
 'NA19036.hap2': 'haplotype2-0000071',
 'HG03732.hap1': 'haplotype1-0000034',
 'HG00358.hap2': 'haplotype2-0000097',
 'HG00358.hap1': 'haplotype1-0000013',
 'NA19239.hap2': 'haplotype2-0000071',
 'HG03456.hap1': 'haplotype1-0000022',
 'HG00512.hap1': 'haplotype1-0000028',
 'NA20847.hap2': 'haplotype2-0000131',
 'HG01596.hap1': 'haplotype1-0000043',
 'HG00171.hap2': 'haplotype2-0000159',
 'NA19239.hap1': 'haplotype1-0000022',
 'HG02282.hap1': 'haplotype1-0000012',
 'NA19705.hap1': 'haploty

In [35]:
copyNumberDF = pd.DataFrame(index=[x for x in set(smnDF['15'])], columns=['SMN1','SMN2','SMN','SERF1A','NAIP','GTF2H2']).fillna(0)

for key in smnData.keys():
    if str(key.replace("_",".")) in copyNumberDF.index:
        for gene in smnData[key]:
            if smnData[key][gene]['Length']=='FL' and smnData[key][gene]['Coordinates'].split(":")[0] == sampleHaplotypeDict[key.replace("_",'.')]:
                copyNumberDF.at[key.replace("_",'.'), "SMN"]+=1
            else:
                continue
    else:
        continue
        
for key in serfData.keys():
    if str(key.replace("_",".")) in copyNumberDF.index:
        for gene in serfData[key]:
            if serfData[key][gene]['Length']=='FL' and serfData[key][gene]['Coordinates'].split(":")[0] == sampleHaplotypeDict[key.replace("_",'.')]:
                copyNumberDF.at[key.replace("_",'.'), "SERF1A"]+=1
            else:
                continue
    else:
        continue
        
for key in naipData.keys():
    if str(key.replace("_",".")) in copyNumberDF.index:
        for gene in naipData[key]:
            if naipData[key][gene]['Length']=='FL' and naipData[key][gene]['Coordinates'].split(":")[0] == sampleHaplotypeDict[key.replace("_",'.')]:
                copyNumberDF.at[key.replace("_",'.'), "NAIP"]+=1
            else:
                continue
    else:
        continue
        
for key in gtfData.keys():
    if str(key.replace("_",".")) in copyNumberDF.index:
        for gene in gtfData[key]:
            if gtfData[key][gene]['Length']=='FL' and gtfData[key][gene]['Coordinates'].split(":")[0] == sampleHaplotypeDict[key.replace("_",'.')]:
                copyNumberDF.at[key.replace("_",'.'), "GTF2H2"]+=1
            else:
                continue
    else:
        continue

In [45]:
naipData['NA20509_hap1']

{'0': {'Length': 'TRUNCATED',
  'Coordinates': 'haplotype1-0000007:71199565-71204244'},
 '1': {'Length': 'TRUNCATED',
  'Coordinates': 'haplotype1-0000007:71215332-71220478'},
 '2': {'Length': 'TRUNCATED',
  'Coordinates': 'haplotype1-0000007:71611964-71647680'},
 '3': {'Length': 'TRUNCATED',
  'Coordinates': 'haplotype1-0000007:72319216-72354897'}}

In [36]:
for sample in copyNumberDF.index:
    testDF = smnDF[smnDF['15']==sample].copy()
    for x in testDF[testDF['8']=='SMN1_EXON1']['16']:
        try:
            copyNumberDF.at[sample,x]+=1
        except:
            print(sample, x)

HG00733.hap2 SMN?
HG00731.hap2 SMN?


In [37]:
davidSMNDF = pd.read_csv('/home/mark/Desktop/SMN/David/chr5_70300000_72100000_completeASMs_withSource.tsv', sep='\t')
hgsvc3David = davidSMNDF[davidSMNDF['source']=='hgsvc3'].copy()
hgsvc3David['Haplotype']=['Hap'+str(x.split("_")[1]) for x in hgsvc3David['asm.id']]
hgsvc3David['SampleName']=[str(x).replace("GM","NA").replace("_",'.hap') for x in hgsvc3David['asm.id']]
hgsvc3David.set_index("SampleName", inplace=True)

In [38]:
hgsvc3David

,asm.id,query.name,revcomp,NucFreq_valid,sample.id,source,Haplotype
SampleName,,,,,,,
NA18989.hap1,GM18989_1,haplotype1-0000016,False,True,GM18989,hgsvc3,Hap1
NA18989.hap2,GM18989_2,haplotype2-0000143,False,True,GM18989,hgsvc3,Hap2
NA19129.hap1,GM19129_1,haplotype1-0000007,False,False,GM19129,hgsvc3,Hap1
NA19129.hap2,GM19129_2,haplotype2-0000088,True,True,GM19129,hgsvc3,Hap2
NA19331.hap1,GM19331_1,haplotype1-0000049,True,True,GM19331,hgsvc3,Hap1
...,...,...,...,...,...,...,...
NA20509.hap2,NA20509_2,haplotype2-0000078,False,True,NA20509,hgsvc3,Hap2
NA20847.hap1,NA20847_1,haplotype1-0000040,True,True,NA20847,hgsvc3,Hap1
NA20847.hap2,NA20847_2,haplotype2-0000131,False,True,NA20847,hgsvc3,Hap2


In [39]:
goodNAIP=[]
for x in copyNumberDF[copyNumberDF['NAIP']>=2].index:
    if x in hgsvc3David.index:
        goodNAIP.append(x)

In [42]:
copyNumberDF.loc['NA20509.hap1']

SMN1      1
SMN2      1
SMN       2
SERF1A    2
NAIP      0
GTF2H2    1
Name: NA20509.hap1, dtype: int64

In [40]:
copyNumberDF.loc[goodNAIP]

,SMN1,SMN2,SMN,SERF1A,NAIP,GTF2H2
NA19331.hap1,2,1,3,3,2,2
NA19239.hap1,1,1,2,2,2,3
NA19036.hap1,2,0,2,2,2,3
HG03456.hap2,2,1,3,3,2,3
HG01505.hap1,1,1,2,2,2,3


In [41]:
#copyNumberDF.to_csv('/home/mark/Desktop/SMN/Dataframes/SMN_CopyNumber_Dataframe_08-05-2024-CLEAN.csv')

In [16]:
#chr5: 71405073-71411378
#sample present : HG00731, HG00733, NA19650, HG00512

In [17]:
smnDF['GeneCompleteness']='NONE'
for row in smnDF.index:
    haplotype = str(smnDF.at[row,'15'])
    contig = str(smnDF.at[row,'3'])
    start  = int(smnDF.at[row,'4'])
    end = int(smnDF.at[row,'5'])

    if 'SMN' in str(smnDF.at[row,'16']):
        for gene in smnData[haplotype.replace(".","_")].keys():
            geneContig = str(smnData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[0])
            genestart = int(smnData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[0])
            geneend = int(smnData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[1])

            if geneContig == contig and start>=genestart and start <=geneend:
                smnDF.at[row,'GeneCompleteness'] = smnData[haplotype.replace(".","_")][gene]['Length']
                smnData[haplotype.replace(".","_")][gene]['GeneName']=str(smnDF.at[row,'16'])
            else:
                continue
        
    elif 'SERF1A' in str(smnDF.at[row,'16']):
        
        for gene in serfData[haplotype.replace(".","_")].keys():
            geneContig = str(serfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[0])
            genestart = int(serfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[0])
            geneend = int(serfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[1])

            if geneContig == contig and start>=genestart and start <=geneend:
                smnDF.at[row,'GeneCompleteness'] = serfData[haplotype.replace(".","_")][gene]['Length']
                serfData[haplotype.replace(".","_")][gene]['GeneName']=str(smnDF.at[row,'16'])
            else:
                continue
        
    
    elif 'NAIP' in str(smnDF.at[row,'16']):
        
        for gene in naipData[haplotype.replace(".","_")].keys():
            geneContig = str(naipData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[0])
            genestart = int(naipData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[0])
            geneend = int(naipData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[1])

            if geneContig == contig and start>=genestart and start <=geneend:
                smnDF.at[row,'GeneCompleteness'] = naipData[haplotype.replace(".","_")][gene]['Length']
                naipData[haplotype.replace(".","_")][gene]['GeneName']=str(smnDF.at[row,'16'])
            else:
                continue
                
    elif 'GTF2H2' in str(smnDF.at[row,'16']):
        
        for gene in gtfData[haplotype.replace(".","_")].keys():
            geneContig = str(gtfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[0])
            genestart = int(gtfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[0])
            geneend = int(gtfData[haplotype.replace(".","_")][gene]['Coordinates'].split(":")[1].split("-")[1])

            if geneContig == contig and start>=genestart and start <=geneend:
                smnDF.at[row,'GeneCompleteness'] = gtfData[haplotype.replace(".","_")][gene]['Length']
                gtfData[haplotype.replace(".","_")][gene]['GeneName']=str(smnDF.at[row,'16'])
            else:
                continue
        
    else:
        continue